# Final Project Features

### Data Processing

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import functools
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson

In [13]:
path_to_trade_data = 'PATH_TO_TRADE_BOOK_DATA'
iNAV_path = 'PATH_TO_INAV_DATA'
where_to_save_data = 'PATH_TO_SAVE_LOCATION'

### Functions

In [14]:
def clean_etf_df(etf_df):
    """take etf dataframe and clean it up."""

    etf_df.drop('SYM_SUFFIX',axis=1,inplace=True)
    etf_df.dropna(inplace=True)

    etf_df['DATE'] = pd.to_datetime(etf_df['DATE'],format='%Y%m%d')
    etf_df['received'] = etf_df['DATE'].astype(str)+' '+etf_df['TIME_M']
    etf_df['received'] = etf_df['received'].apply(pd.Timestamp)

    etf_df['bid_ask_spread'] = etf_df['NBO']-etf_df['NBB']
    etf_df['bid_ask_over_price'] = etf_df['bid_ask_spread']/etf_df['PRICE']
    etf_df['bid_ask_over_price_timestamp_sum'] = etf_df.groupby('received')['bid_ask_over_price'].transform(sum)
    etf_df['bid_ask_spread_timestamp_sum'] = etf_df.groupby('received')['bid_ask_spread'].transform(sum)
    etf_df['dummy'] =1
    etf_df['timestamp_count'] = etf_df.groupby('received')['dummy'].transform(sum) # trades per timestamp
    etf_df['timestamp_volume'] = etf_df.groupby('received')['SIZE'].transform(sum) # volume in timestamp

    etf_df.sort_values(['received','TR_SEQNUM'],inplace=True)
    etf_df['cumulative_trade_count'] = etf_df['dummy'].cumsum()
    etf_df['cumulative_volume'] = etf_df['SIZE'].cumsum() # cumulative volume

    for i in [1,2]:  # 1 lag, then 2 lag
        where0 = (etf_df.LeeReady==0)
        lag = etf_df.shift(i)
        # curr price is lower than prev price (seller initiated (-1))
        etf_df[f"lag{i}"] = (etf_df[where0].PRICE < lag[where0].PRICE) * -1
        # curr price is higher than prev price (buyer initiated (+1))
        etf_df[f"lag{i}"] += (etf_df[where0].PRICE > lag[where0].PRICE) * 1
        etf_df[f"lag{i}"] = etf_df[f"lag{i}"].fillna(0)

        etf_df.LeeReady += etf_df[f"lag{i}"]

    etf_df = etf_df.drop(columns=[f"lag{i}"])

    etf_df['direction_size'] = etf_df['SIZE']*etf_df['LeeReady']
    etf_df['direction_size'] = etf_df.groupby('received')['direction_size'].transform(sum) # sum per timestamp
    
    etf_df['dollar_direction'] = etf_df['PRICE']*etf_df['SIZE']*etf_df['LeeReady']
    etf_df['dollar_direction'] = etf_df.groupby('received')['dollar_direction'].transform(sum) # sum per timestamp
    
    etf_df = etf_df.groupby('received').tail(1) # take last trade per timestamp
    etf_df.set_index('received',inplace=True) # set index to received timestamp

    time_criteria = (((etf_df.index.hour >= 9)&(etf_df.index.minute >= 30))|
                     (etf_df.index.hour>9))&(etf_df.index.hour < 16)

    etf_df = etf_df.loc[time_criteria].sort_index()
    
    etf_df['order_imbalance'] = (etf_df['NBBqty']-etf_df['NBOqty'])/(etf_df['NBBqty']+etf_df['NBOqty'])
    
    return etf_df

In [15]:
def tau_sum(data_series,Tau):
    """Find cumulative sum fo values in the prior Tau time interval."""
    cumulative_sum = data_series.cumsum()
    
    T_cumulative_sum = cumulative_sum.reindex(cumulative_sum.index-pd.Timedelta(Tau), method='bfill')
    T_cumulative_sum.index = cumulative_sum.index

    sum_in_period = cumulative_sum-T_cumulative_sum
    
    return sum_in_period

def tau_average(field,trade_counts,Tau):
    field_cum = field.cumsum()
    T_field_cum = field_cum.reindex(field_cum.index-pd.Timedelta(Tau), method='bfill')
    T_field_cum.index = field_cum.index

    field_sum_in_period = field_cum-T_field_cum
    trades_in_period = tau_sum(trade_counts,Tau)

    tau_mean_field = field_sum_in_period/trades_in_period
    
    return tau_mean_field

In [16]:
def T_ewm_vol(price_series,T):
    """Calculate cum log returns up to and not including T.
        EWMVar: Mean squared T return with decay = T.
        Vol = square root of EWMVar."""
    #avoid jumps in the return series from close to open (could remove if we want to include these)
    new_day = np.where(price_series.reset_index()['received'].dt.day.diff()!=0,np.nan,1)
    returns = np.log(price_series/price_series.shift()) * new_day
    T_returns = returns.rolling(T,closed='left').sum()
    
    ewm_var = (T_returns**2).ewm(halflife = T, times=returns.index,ignore_na=True).mean()
    ewm_vol = np.sqrt(ewm_var)
    ewm_vol.name = 'exp weighted volatility'
    return ewm_vol

def calc_volumes(trade_sizes):
    """Calculate total volume for Tau time period"""
    
    volumes = pd.concat(
        {'volume_1min':tau_sum(trade_sizes,'60s'),
         'volume_2min':tau_sum(trade_sizes,'120s'),
         'volume_4min':tau_sum(trade_sizes,'240s'),
         'volume_5min':tau_sum(trade_sizes,'300s'),
         'volume_10min':tau_sum(trade_sizes,'600s'),
         'volume_15min':tau_sum(trade_sizes,'900s'),
         'volume_30min':tau_sum(trade_sizes,'1800s')},
        axis=1)

    return volumes

def calc_tau_vwap(trade_prices,trade_sizes, Tau):
    """calculate volume weighted average price over tau time period"""
    tau_volume = tau_sum(trade_sizes,Tau)
    tau_total_dollar_size = tau_sum(trade_prices*trade_sizes,Tau)
    
    vwap = tau_total_dollar_size/tau_volume
    
    return vwap


def T_fwd_rtn(price_series,T):
    """Calculate T forward returns"""

    T_fwd_prices = price_series.reindex(price_series.index+pd.Timedelta(T), method='ffill')
    T_fwd_prices.index = price_series.index
    T_fwd_rtns = T_fwd_prices/price_series-1

    return T_fwd_rtns

def fwd_volume(trade_sizes,Tau):
    """calculate vwap for next Tau time period"""
    trade_sizes_sum = trade_sizes.cumsum()

    T_trade_sizes_sum = trade_sizes_sum.reindex(trade_sizes_sum.index+pd.Timedelta(Tau), method='ffill')
    T_trade_sizes_sum.index = trade_sizes_sum.index
    T_fwd_total_volume = (T_trade_sizes_sum-trade_sizes_sum) # if there were no trades, take previous price

    T_fwd_total_volume.index = trade_sizes_sum.index
    
    return T_fwd_total_volume



def fwd_vwap(trade_prices, trade_sizes,Tau):
    """calculate vwap for next Tau time period"""
    trade_sizes_sum = trade_sizes.cumsum()

    T_trade_sizes_sum = trade_sizes_sum.reindex(trade_sizes_sum.index+pd.Timedelta(Tau), method='ffill')
    T_trade_sizes_sum.index = trade_sizes_sum.index
    T_fwd_total_volume = (T_trade_sizes_sum-trade_sizes_sum) # if there were no trades, take previous price

    price_times_size = trade_prices*trade_sizes
    price_times_size_sum = price_times_size.cumsum()

    T_price_times_size_sum = price_times_size_sum.reindex(price_times_size_sum.index+pd.Timedelta(Tau), method='ffill')
    T_price_times_size_sum.index = price_times_size_sum.index
    T_price_times_size_sum_total = T_price_times_size_sum - price_times_size_sum

    fwd_vwap = T_price_times_size_sum_total/T_fwd_total_volume
    
    return fwd_vwap


def vwap_fwd_rtn(trade_prices, trade_sizes, T, vwap_tau):
    """calculate vwap forward return"""
    
    T_fwd_prices = trade_prices.reindex(trade_prices.index+pd.Timedelta(T), method='ffill')
    T_fwd_sizes = trade_sizes.reindex(trade_sizes.index+pd.Timedelta(T), method='ffill')
    
    T_fwd_vwap = fwd_vwap(trade_prices = T_fwd_prices, trade_sizes = T_fwd_sizes, Tau = vwap_tau) # fill with previous price
    T_fwd_vwap = T_fwd_vwap.fillna(method='ffill')
    
    T_fwd_vwap.index = trade_prices.index
    
    start_vwap = fwd_vwap(trade_prices = trade_prices, trade_sizes=trade_sizes,Tau = vwap_tau) # fill with previous price
    start_vwap = start_vwap.fillna(method='ffill')
    
    
    vwap_fwd_rtns = T_fwd_vwap/start_vwap-1
    
    return vwap_fwd_rtns

In [17]:
def calc_fwd_rtns(etf_trades):
    "Calculate different forward return periods"
    fwd_rtns = pd.concat(
        {'fwd_rtn_5min':T_fwd_rtn(etf_trades['PRICE'],'300s'),
         'fwd_rtn_10min':T_fwd_rtn(etf_trades['PRICE'],'600s'),
         'fwd_rtn_15min':T_fwd_rtn(etf_trades['PRICE'],'900s')},
        axis=1)
    
    return fwd_rtns

In [18]:
def calc_ewm_vol(price_series):
    """Calculate EWMA Volatility Metrics"""
    
    intervals = [1,2,4,5,15,20,25,30,45,60]
    times = [str(60*t)+'s' for t in intervals]
    vols = pd.DataFrame(index=price_series.index)
    for time,interval in zip(times,intervals):
        annualize = 6.5*(60/interval)*252
        vols['ewm_vol_'+time] = np.sqrt(T_ewm_vol(price_series,time)**2*annualize).replace(0,np.nan)
    
    return vols

In [19]:
def calc_flow_metrics(sized_directions,dollar_sized_directions):
    """Calculate Flow Metrics"""
    
    dollar_flows = pd.concat(
        {'dollar_flow_1min':tau_sum(dollar_sized_directions,'60s'),
         'dollar_flow_2min':tau_sum(dollar_sized_directions,'120s'),
         'dollar_flow_4min':tau_sum(dollar_sized_directions,'240s'),
         'dollar_flow_5min':tau_sum(dollar_sized_directions,'300s'),
         'dollar_flow_15min':tau_sum(dollar_sized_directions,'900s'),
         'dollar_flow_20min':tau_sum(dollar_sized_directions,'1200s'),
         'dollar_flow_25min':tau_sum(dollar_sized_directions,'1500s'),
         'dollar_flow_30min':tau_sum(dollar_sized_directions,'1800s'),
         'dollar_flow_45min':tau_sum(dollar_sized_directions,'2700s'),
         'dollar_flow_60min':tau_sum(dollar_sized_directions,'3600s')},
        axis=1)
    
    flows = pd.concat(
        {'flow_1min':tau_sum(sized_directions,'60s'),
         'flow_2min':tau_sum(sized_directions,'120s'),
         'flow_4min':tau_sum(sized_directions,'240s'),
         'flow_5min':tau_sum(sized_directions,'300s'),
         'flow_15min':tau_sum(sized_directions,'900s'),
         'flow_20min':tau_sum(sized_directions,'1200s'),
         'flow_25min':tau_sum(sized_directions,'1500s'),
         'flow_30min':tau_sum(sized_directions,'1800s'),
         'flow_45min':tau_sum(sized_directions,'2700s'),
         'flow_60min':tau_sum(sized_directions,'3600s')},
        axis=1)

    EWMA_flows = pd.concat(
        {'flow_1min_EWMA':flows['flow_1min'].ewm(halflife='120s',times=flows.index).mean(),
         'flow_2min_EWMA':flows['flow_2min'].ewm(halflife='240s',times=flows.index).mean(),
         'flow_4min_EWMA':flows['flow_4min'].ewm(halflife='480s',times=flows.index).mean(),
         'flow_5min_EWMA':flows['flow_5min'].ewm(halflife='600s',times=flows.index).mean(),
         'flow_15min_EWMA':flows['flow_15min'].ewm(halflife='1800s',times=flows.index).mean(),
         'flow_20min_EWMA':flows['flow_20min'].ewm(halflife='2400s',times=flows.index).mean(),
         'flow_25min_EWMA':flows['flow_25min'].ewm(halflife='3000s',times=flows.index).mean(),
         'flow_30min_EWMA':flows['flow_30min'].ewm(halflife='3600s',times=flows.index).mean(),
         'flow_45min_EWMA':flows['flow_45min'].ewm(halflife='5400s',times=flows.index).mean(),
         'flow_60min_EWMA':flows['flow_60min'].ewm(halflife='7200s',times=flows.index).mean()},
        axis=1)

    all_flows = flows.join([EWMA_flows,dollar_flows])
    
    return all_flows

In [20]:
def calc_bid_ask_metrics(bid_ask_spread_sum, bid_ask_over_price_sum, trade_counts):
    """Calculate average bid ask spread and average bid ask spread divided by price variables"""
    bid_ask = pd.concat(
        {'bid_ask_1min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='60s'),
         'bid_ask_2min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='120s'),
         'bid_ask_5min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='300s'),
         'bid_ask_10min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='600s'),
         'bid_ask_15min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='900s'),
         'bid_ask_30min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='1800s'),
         'bid_ask_60min':tau_average(field=bid_ask_spread_sum, trade_counts=trade_counts, Tau='3600s')},
    axis=1) 
    
    bid_ask_prices = pd.concat(
        {'bid_ask_price_1min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='60s'),
         'bid_ask_price_2min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='120s'),
         'bid_ask_price_5min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='300s'),
         'bid_ask_price_10min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='600s'),
         'bid_ask_price_15min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='900s'),
         'bid_ask_price_30min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='1800s'),
         'bid_ask_price_60min':tau_average(field=bid_ask_over_price_sum,trade_counts=trade_counts,Tau='3600s')},
        axis=1)

    bid_ask_met = bid_ask_prices.join(bid_ask)
    
    return bid_ask_met

In [21]:
def calc_imbalance_metrics(imbalances, trade_counts):
    """Calculate tau time period average order imbalances. for 1,2,4,5,10,15,30 minutes"""
    
    order_imbalances = pd.concat(
        {'order_imbalance_1min':tau_average(imbalances, trade_counts, '60s'),
         'order_imbalance_2min':tau_average(imbalances, trade_counts, '120s'),
         'order_imbalance_4min':tau_average(imbalances, trade_counts, '240s'),
         'order_imbalance_5min':tau_average(imbalances, trade_counts, '300s'),
         'order_imbalance_10min':tau_average(imbalances, trade_counts, '600s'),
         'order_imbalance_15min':tau_average(imbalances, trade_counts, '900s'),
         'order_imbalance_30min':tau_average(imbalances, trade_counts, '1800s')},
        axis=1)
    
    return order_imbalances

In [22]:
def calc_all_metrics(etf_trades, iNAV):
    """Calculate all the trade data metrics, forward returns, and join it together."""
    etf_trades = clean_etf_df(etf_trades)

    flow_metrics = calc_flow_metrics(sized_directions = etf_trades['direction_size'],
                                     dollar_sized_directions = etf_trades['dollar_direction'])

    bid_ask_metrics = calc_bid_ask_metrics(bid_ask_spread_sum = etf_trades['bid_ask_spread_timestamp_sum'],
                                           bid_ask_over_price_sum = etf_trades['bid_ask_over_price_timestamp_sum'],
                                           trade_counts = etf_trades['timestamp_count'])
    
    volatilities = calc_ewm_vol(price_series = etf_trades['PRICE'])
    volumes = calc_volumes(trade_sizes = etf_trades['timestamp_volume'])
    order_imbalances = calc_imbalance_metrics(etf_trades['order_imbalance'], trade_counts=etf_trades['timestamp_count'])
    
    fields_from_trade_book = etf_trades.loc[:,['PRICE','NBB','NBO','NBOqty','NBBqty',
                                               'cumulative_trade_count','cumulative_volume','order_imbalance']]
    
    
    # join all the fields from trade book together
    trade_book_variables = flow_metrics.join([bid_ask_metrics,
                                              volatilities,
                                              volumes,
                                              order_imbalances,
                                              fields_from_trade_book])
    
    independent_variables = pd.merge_asof(iNAV,
                                          trade_book_variables,
                                          left_index=True,
                                          right_index=True,
                                          direction='backward',
                                          allow_exact_matches=True)

    independent_variables['nav_discount_bid'] = independent_variables['NBB']/independent_variables['iNAV']-1
    independent_variables['nav_discount_ask'] = independent_variables['NBO']/independent_variables['iNAV']-1
    
    # forward returns
    fwd_rtns = calc_fwd_rtns(etf_trades)
    
    vwap_fwd_rtns = pd.concat(
       {'vwap_fwd_rtn_5min_10s_vwap':vwap_fwd_rtn(trade_prices=etf_trades['PRICE'],
                                                  trade_sizes = etf_trades['timestamp_volume'],
                                                  T='600s',
                                                  vwap_tau = '10s'),
       'fwd_vwap_10s':fwd_vwap(trade_prices = etf_trades['PRICE'], 
                                 trade_sizes=etf_trades['timestamp_volume'],
                                 Tau = '10s'),
       'fwd_volume_10s':fwd_volume(trade_sizes=etf_trades['timestamp_volume'],
                                   Tau = '10s')},
       axis=1)
    
    all_fwd_rtns = fwd_rtns.join(vwap_fwd_rtns)
    all_fwd_rtns.index = all_fwd_rtns.index - pd.Timedelta(50,'milli') # subtract 50 milliseconds for latency

    all_vars = pd.merge_asof(independent_variables,
                             all_fwd_rtns,
                             left_index=True,
                             right_index=True,
                             direction='forward',
                             allow_exact_matches=True)
    
    return all_vars 

#### HYG

In [33]:
iNAV = pd.read_csv(iNAV_path+'hygiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'HYG_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'hyg_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'HYG_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'hyg_metrics_2021.csv')

del vars_2021

#### JNK

In [38]:
iNAV = pd.read_csv(iNAV_path+'jnkiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'JNK_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'jnk_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'JNK_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'jnk_metrics_2021.csv')

del vars_2021

#### BKLN

In [39]:
iNAV = pd.read_csv(iNAV_path+'bklniv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'BKLN_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'bkln_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'BKLN_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'bkln_metrics_2021.csv')

del vars_2021

#### SRLN

In [40]:
iNAV = pd.read_csv(iNAV_path+'srlniv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'SRLN_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'srln_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'SRLN_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'srln_metrics_2021.csv')

del vars_2021

#### PFF

In [41]:
iNAV = pd.read_csv(iNAV_path+'pffiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'PFF_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'pff_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'PFF_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'pff_metrics_2021.csv')

del vars_2021

#### PGX

In [42]:
iNAV = pd.read_csv(iNAV_path+'pgxiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'PGX_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'pgx_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'PGX_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'pgx_metrics_2021.csv')

del vars_2021

#### SPHY

In [43]:
iNAV = pd.read_csv(iNAV_path+'sphyiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'SPHY_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'sphy_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'SPHY_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'sphy_metrics_2021.csv')

del vars_2021

#### HYGH

In [44]:
iNAV = pd.read_csv(iNAV_path+'hyghiv.csv', index_col = 'date',parse_dates=True).sort_index()

vars_1920 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'HYGH_1920.csv'),
                            iNAV=iNAV.loc[iNAV.index.year < 2021].copy())

vars_1920.to_csv(where_to_save_data+'hygh_metrics_1920.csv')

del vars_1920

vars_2021 = calc_all_metrics(etf_trades = pd.read_csv(path_to_trade_data+'HYGH_2021.csv'),
                            iNAV=iNAV.loc[iNAV.index.year == 2021].copy())

vars_2021.to_csv(where_to_save_data+'hygh_metrics_2021.csv')

del vars_2021